In [ ]:
import logging
import logging.config
import os
import json

from googleapiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools
from oauth2client.file import Storage

APP_SHORT_NAME = 'TeamDriveFolders'

In [ ]:
import mymod

In [ ]:
def setup_logging(
    default_path='logging.json',
    default_level=logging.INFO,
    env_key='LOG_CFG'
):
    """Setup logging configuration

    """
    path = default_path
    value = os.getenv(env_key, None)
    if value:
        path = value
    if os.path.exists(path):
        with open(path, 'rt') as f:
            config = json.load(f)
        logging.config.dictConfig(config)
    else:
        logging.basicConfig(level=default_level)

In [ ]:
def main(): 
    logger = logging.getLogger(__name__)
    setup_logging()

    logger.info('It works!')

    rover = mymod.dog()
    rover.sit()
    mymod.foobar()
    
    
    
    
    
    print (2-2)
if __name__ == '__main__':
    main()